# Generar rutas de transporte

In [1]:
import pandas as pd
from math import radians, sin, cos, sqrt, atan2
import random
from GrafoDestinos import GrafoDestinos

random.seed(42)

In [2]:
df_destinos = pd.read_csv('../DestinosConAeropuertos/destinos_tiene_aeropuerto_column.csv')
df_empresas = pd.read_csv('../Empresas/empresas_transporte.csv')

In [3]:
df_aerolineas = df_empresas[df_empresas['tipo'] == 'Aerolínea']
df_autobuses = df_empresas[df_empresas['tipo'] == 'Autobuses']

In [4]:
grafo_destinos = GrafoDestinos('../DestinosConAeropuertos/destinos_tiene_aeropuerto_column.csv', 20, 1, 0.8)

In [5]:
def generar_ruta_transporte(origin, target, org_id, tar_id):
    dist = grafo_destinos.obtener_distancia_entre_destinos(origin, target)

    if origin['tiene_aeropuerto'] and target['tiene_aeropuerto']:
        # Ruta aérea
        duracion_min = int(dist / 800 * 60 + 30)  # vuelo + espera
        precio = int(dist * 1.8 + 10000)
        empresa = df_aerolineas.sample(n=1).iloc[0]
        rutas.append({
            "origen_id": org_id,
            "target_id": tar_id,
            "empresa_id": empresa['id'],
            "origen": origin['nombre'],
            "destino": target['nombre'],
            "tipo": "Avión",
            "empresa": empresa['nombre'],
            "distancia_km": round(dist, 2),
            "duracion_min": duracion_min,
            "precio": precio
        })

    # Ruta terrestre (disponible siempre)
    duracion_min = int(dist / 60 * 60 + 60)  # velocidad media + paradas
    precio = int(dist * 0.7 + 100)
    empresa = df_autobuses.sample(n=1).iloc[0]
    rutas.append({
        "origen_id": org_id,
        "target_id": tar_id,
        "empresa_id": empresa['id'],
        "origen": origin['nombre'],
        "destino": target['nombre'],
        "tipo": "Autobús",
        "empresa": empresa['nombre'],
        "distancia_km": round(dist, 2),
        "duracion_min": duracion_min,
        "precio": precio
    })

In [6]:
rutas = list()
for i in range(len(grafo_destinos.matriz_adyacencia)):
    origin = df_destinos.iloc[i]
    for j in range(len(grafo_destinos.matriz_adyacencia[i])):
        conexion = grafo_destinos.matriz_adyacencia[i][j]
        if conexion != 0:
            target = df_destinos.iloc[j]
            generar_ruta_transporte(origin, target, i+1, j+1)

In [7]:
df_rutas = pd.DataFrame(rutas)
df_rutas['id'] = [i+1 for i in range(len(df_rutas))]
df_rutas

,origen_id,target_id,empresa_id,origen,destino,tipo,empresa,distancia_km,duracion_min,precio,id
0,1,2,3,Cabo San Lucas,Cancún,Avión,Viva Aerobus,2384.30,208,14291,1
1,1,2,6,Cabo San Lucas,Cancún,Autobús,ADO,2384.30,2444,1769,2
2,1,3,1,Cabo San Lucas,Loreto,Avión,Aeroméxico,375.29,58,10675,3
3,1,3,7,Cabo San Lucas,Loreto,Autobús,Estrella Blanca,375.29,435,362,4
4,1,4,1,Cabo San Lucas,San José del Cabo,Avión,Aeroméxico,28.92,32,10052,5
...,...,...,...,...,...,...,...,...,...,...,...
17094,225,216,8,"Terrenos del Lote ""Cicuaque""",Huejutla de Reyes,Autobús,Futura,82.82,142,157,17095
17095,225,221,8,"Terrenos del Lote ""Cicuaque""","Terrenos del Lote ""La Asunción""",Autobús,Futura,25.53,85,117,17096
17096,225,222,8,"Terrenos del Lote ""Cicuaque""","Terrenos del Lote ""La Isla""",Autobús,Futura,35.79,95,125,17097
17097,225,223,8,"Terrenos del Lote ""Cicuaque""",Tuxpan,Autobús,Futura,26.99,86,118,17098


In [8]:
df_rutas.to_csv('./rutas_entre_destinos.csv', index=False)